# Iris data set - Keras
We goning to demonstrate machine learning prediction on Iris data set. We gonna create model that will predict species of Iris flower by sepal width, sepal length, petal width, and petal length.

## Data set
[Iris data set](http://mlr.cs.umass.edu/ml/machine-learning-databases/iris/iris.data) is in the cvs format.

### Read set and process
We gonna use [pandas](http://pandas.pydata.org/) python library to process raw data.

In [1]:
import pandas as pd

iris_data = pd.read_csv('data/iris.csv')
iris_data.head()

,sepal_length,sepal_width,petal_length,petal_width,type
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


Pandas library has many nice features where you can specify condions

In [2]:
iris_data.loc[(iris_data['type'] == 
               'Iris-versicolor') & (iris_data['sepal_length'] < 5.0)]

,sepal_length,sepal_width,petal_length,petal_width,type
57,4.9,2.4,3.3,1.0,Iris-versicolor


We can extract the data in this format from pandas like this:

In [3]:
all_inputs = iris_data[['sepal_length', 'sepal_width',
                             'petal_length', 'petal_width']].values

Similarly, we can extract the types of Iris flower

In [4]:
all_types = iris_data['type'].values

Make sure that you don't mix up the order of the entries all_inputs[5] inputs should correspond to the type in all_types[5]

In [5]:
print(all_inputs[5]," - ", all_types[5])

[ 5.4  3.9  1.7  0.4]  -  Iris-setosa


We convert all the Iris flower types from data set into respective numbers using [numpy](http://www.numpy.org/) library

In [6]:
import numpy as np
outputs_vals, outputs_ints = np.unique(all_types, return_inverse=True)

Now we split data set into training and testing sets

In [7]:
inds = np.random.permutation(len(all_inputs))
train_inds, test_inds = np.array_split(inds, 2)

Also we convert output strings (Iris types) into integers

In [8]:
import keras as kr
outputs_vals, outputs_ints = np.unique(all_types, return_inverse=True)
outputs_cats = kr.utils.to_categorical(outputs_ints)

Using TensorFlow backend.


And now we populate the training and testing arrays

In [9]:
inputs_train, outputs_train = all_inputs[train_inds], outputs_cats[train_inds]
inputs_test,  outputs_test  = all_inputs[test_inds],  outputs_cats[test_inds]

For the actual creation of the machine learning traing, model and testing we going to use [keras](https://keras.io/).  
We start by creating the neural network model.

In [10]:
model = kr.models.Sequential()
model.add(kr.layers.Dense(16, input_shape=(4,)))
model.add(kr.layers.Activation("sigmoid"))
model.add(kr.layers.Dense(3))
model.add(kr.layers.Activation("softmax"))

 - Here we created sequential model with input layer which takes 4 inputs  
 - Next layer consist of 16 neurons to process the 4 inputs
 - Next layer is using sigmoid function to fit out between -1 and 1.
 - And finally we are using softmax function to highlight the one dominant output out of all

### Now we configure training model
We use adam optimizer and categorical cross entropy as the loss function

In [11]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

### Training of the model
Now we traing the model with training data. We can specify epochs, which is a number of training runs we want to train our neural network with. Also we can specify batch size, which determins number of tests after which our model adjust itself. 

In [12]:
model.fit(inputs_train, outputs_train, epochs=100, batch_size=1, verbose=1)

Epoch 1/100
75/75 [==============================] - 1s 7ms/step - loss: 1.2309 - acc: 0.3467
Epoch 2/100
75/75 [==============================] - 0s 2ms/step - loss: 1.0938 - acc: 0.3600
Epoch 3/100
75/75 [==============================] - 0s 2ms/step - loss: 1.0355 - acc: 0.4800
Epoch 4/100
75/75 [==============================] - 0s 2ms/step - loss: 0.9984 - acc: 0.5867
Epoch 5/100
75/75 [==============================] - 0s 2ms/step - loss: 0.9648 - acc: 0.6800
Epoch 6/100
75/75 [==============================] - 0s 2ms/step - loss: 0.9150 - acc: 0.7333
Epoch 7/100
75/75 [==============================] - 0s 2ms/step - loss: 0.8626 - acc: 0.7600
Epoch 8/100
75/75 [==============================] - 0s 2ms/step - loss: 0.8105 - acc: 0.6800
Epoch 9/100
75/75 [==============================] - 0s 2ms/step - loss: 0.7656 - acc: 0.7867
Epoch 10/100
75/75 [==============================] - 0s 2ms/step - loss: 0.7271 - acc: 0.6933
Epoch 11/100
75/75 [==============================] - 0s 2m

Epoch 87/100
75/75 [==============================] - 0s 2ms/step - loss: 0.1434 - acc: 0.9733
Epoch 88/100
75/75 [==============================] - 0s 2ms/step - loss: 0.1413 - acc: 0.9867
Epoch 89/100
75/75 [==============================] - 0s 2ms/step - loss: 0.1414 - acc: 0.9867
Epoch 90/100
75/75 [==============================] - 0s 2ms/step - loss: 0.1358 - acc: 0.9867
Epoch 91/100
75/75 [==============================] - 0s 2ms/step - loss: 0.1352 - acc: 0.9733
Epoch 92/100
75/75 [==============================] - 0s 2ms/step - loss: 0.1315 - acc: 0.9867
Epoch 93/100
75/75 [==============================] - 0s 2ms/step - loss: 0.1318 - acc: 0.9867
Epoch 94/100
75/75 [==============================] - 0s 2ms/step - loss: 0.1278 - acc: 0.9867
Epoch 95/100
75/75 [==============================] - 0s 2ms/step - loss: 0.1284 - acc: 0.9867
Epoch 96/100
75/75 [==============================] - 0s 2ms/step - loss: 0.1257 - acc: 0.9733
Epoch 97/100
75/75 [==============================

### Evaluation of the model
Here we run evaluation on the model we just trained with test data.

In [13]:
loss, accuracy = model.evaluate(inputs_test, outputs_test, verbose=1)

print("\n\nLoss: %10.5f\tAccuracy: %2.2f %%" % (loss, accuracy*100))

75/75 [==============================] - 0s 305us/step


Loss:    0.12099	Accuracy: 96.00 %


### Model test
And finally we going to test a model we created and trained

In [14]:
prediction = np.around(model.predict(np.expand_dims(inputs_test[24], axis=0))).astype(np.int)[0]
print("Model prediction in hot vector output: %s" % prediction)
print("Model prediction in actual Iris type: %s" % outputs_vals[prediction.astype(np.bool)][0])
print("Actual Iris flower type in data set: Vector: %s Type: %s" % (outputs_test[24].astype(np.int), outputs_vals[prediction.astype(np.bool)][0]))

Model prediction in hot vector output: [0 1 0]
Model prediction in actual Iris type: Iris-versicolor
Actual Iris flower type in data set: Vector: [0 1 0] Type: Iris-versicolor


### Conclusion
As we can see prediction model is behaving as expected and gives us correct prediction with almost 99% perfect accuracy. We can save this model and load and use it for futher predictions later as the model is trained already.

In [1]:
model.save("iris_model.h5")

NameError: name 'model' is not defined

Adapted from: -  `https://github.com/salmanahmad4u/keras-iris/blob/master/iris_nn.py`  
 - `https://github.com/emerging-technologies/keras-iris`